In [1]:
import findspark
findspark.init('/home/kalyan/spark-2.1.0-bin-hadoop2.7')
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Linear Regression").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
# Create instance of our model
lr = LinearRegression(featuresCol="features", labelCol='label', predictionCol='prediction')

In [8]:
lrModel = lr.fit(training)

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
training_summary = lrModel.summary

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [17]:
training_data, test_data = all_data.randomSplit([0.7, 0.3])

In [19]:
training_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                325|
|   mean|-0.6000688338184579|
| stddev| 10.854622161498241|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [21]:
correct_model = lr.fit(training_data)

In [22]:
test_results = correct_model.evaluate(test_data)

In [23]:
test_results.rootMeanSquaredError

9.69040412229198

In [24]:
unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
predictions = correct_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.7390646407111836|
|(10,[0,1,2,3,4,5,...| -4.1216644784467915|
|(10,[0,1,2,3,4,5,...|  -1.199586311008554|
|(10,[0,1,2,3,4,5,...| -3.4719868170286228|
|(10,[0,1,2,3,4,5,...|  0.7708838187041408|
|(10,[0,1,2,3,4,5,...| -2.6271145305329764|
|(10,[0,1,2,3,4,5,...|-0.43918234854738925|
|(10,[0,1,2,3,4,5,...|   -4.07169927772393|
|(10,[0,1,2,3,4,5,...|   2.652433291880359|
|(10,[0,1,2,3,4,5,...| -1.2037844418851484|
|(10,[0,1,2,3,4,5,...|  -1.765012039984898|
|(10,[0,1,2,3,4,5,...|  -1.320856167393952|
|(10,[0,1,2,3,4,5,...| -3.7812309612120587|
|(10,[0,1,2,3,4,5,...|  -6.407521135995651|
|(10,[0,1,2,3,4,5,...| -0.9339792898024052|
|(10,[0,1,2,3,4,5,...| -1.7969355047113091|
|(10,[0,1,2,3,4,5,...|  -4.453506129797309|
|(10,[0,1,2,3,4,5,...| -4.5902004723277825|
|(10,[0,1,2,3,4,5,...|-0.27493618574510126|
|(10,[0,1,2,3,4,5,...|  1.434158